Author: Tim Liu  
_To comment better, this series will be in jupyter notebook format._  
_Animated result can be seen after running the .py file_  
_The code is in python, while C++ implementation will also be mentioned._  


BFS is a very basic search-based path finding algorithm
It **uniformly** explores nodes in all directions. And the idea of keeping track of the **_frontier_** and the **_parent node (history)_** is widely applied. 

In [ ]:
import queue
from utils import env, plotting
import time

Specifically, the frontier should be a **FIFO** (first in first out) model. Thus, use a queue.Queue() to represent it in python. In C++, std::queue can achieve the same effect.  
We name a variable "parent" to track the previous node of the current exploring node. This is done with a **dictionary** in python, as it keeps the dual pair conveniently. In C++, **Hash structures** are feasible to keep the pair, like std::unordered_map.  

In [ ]:
class BFS():
    def __init__(self) -> None:
        '''
        BFS maintains a frontier
        after exploring the head`s neighbour, the head node goes to another collection
        thus, frontier is a queue

        To generate the path, we need to record the node`s parent (where it comes from)
        in python, use a dict to track
        '''
        self.frontier = queue.Queue()
        self.parent = dict()

        self.env = env.Env()
        self.start = (3, 3)
        self.goal = (self.env.x_size - 2, self.env.y_size - 2)

        self.path = []

In /utils/env.py, we set up the enviorment of a grid space, with fixed size and obstacle positions.  

In [ ]:
class BFS(BFS):
    def get_neighbor(self, s):
        """
        find reachable neighbors of state s that not in obstacles.
        :param s: state
        :return: neighbors
        """
        # return [(s[0] + u[0], s[1] + u[1]) for u in self.env.motions]
        l = list()
        for move in self.env.motions:
            if (s[0] + move[0], s[1] + move[1]) not in self.env.obs:
                l.append((s[0] + move[0], s[1] + move[1]))
        return l

Oh boy, here comes the majority of the algorithm.  
Essentially, the BFS take the head of the frontier (a queue), and try out its neighbours without any bias.  

If the neighbouring node has not been reached, it is added to the queue as it is to be **explored in the next iteration**. Meanwhile, the parent dictionary should add the pair, now that this neighbouring node **comes from** the current queue`s head.  

And how do we determine whether a node has been explored? We simply check whether this node exists in the parent dictionary **(as the key, not the value)**

In [ ]:
class BFS(BFS):
    def search(self):
        # init with pushing the starting node
        found = False
        self.frontier.put(self.start)
        self.parent[self.start] = None # the starting point`s parent can be None
        while not self.frontier.empty():
            # fetch the queue`s head
            cur_node = self.frontier.get()
            if cur_node == self.goal:
                # self.parent[self.goal] = 
                # self.get_path()
                found = True
                break
            for new_node in self.get_neighbor(cur_node): # get adjacent none-obstacle grid
                if new_node not in self.parent: # this node has not been explored
                    # update frontier and parent
                    self.frontier.put(new_node)
                    self.parent[new_node] = cur_node
        if found:
            self.get_path()
            return self.path, self.parent
        else:
            return None, None

Now that the frontier has reached the goal, we can reconstruct the path bia traversing the parent collection.  
We do this based on the parent dictionary.  

In [ ]:
class BFS(BFS):
    def get_path(self):        
        cur = self.goal
        while cur != self.start:
            self.path.append(cur)
            cur = self.parent[cur] # iterate until reaching the start
        self.path.reverse() #! note that the constructed path is from goal to start, in reverse order
        print("path found!")

In [26]:
def main():
    planner = BFS()
    t_start = time.time()
    path, visited = planner.search();
    t_end = time.time();
    print(t_end - t_start)

if __name__=="__main__":
    main()

path found!
0.017005205154418945
